<center>

## <font color='darkblue'>ASTR 21100/ASTR 31200
<center>

### <font color='darkblue'>"Computational Techniques in Astrophysics"
    
<center>
    
### <font color='darkblue'> Homework 1 (for graduate students): Integrating particle orbits in $N$-body problems

<center>
    
### <font color='darkblue'> 35 points + 16 possible extra-credit

<center>
    
### <font color='darkblue'> Due Wednesday,  Apr 6, 10pm


### <font color='darkblue'>1  (25 points).  Implementing code for $N$-body integration

**1a (20 points).** Implement a *direct N-body code* which takes initial coordinates and velocities of $N$ particles and evolves them forward using a specified constant time step for some input number of steps. At each step and for each particle compute acceleration from other particles. After acceleration is computed, implement update of coordinates and velocities using either 1st order Euler scheme or the 2nd order symplectic integrator (KDK or DKD, see S 2.3.12  in the <a href="https://astro.uchicago.edu/~andrey/classes/a330/notes.pdf">notes</a>) to complete the step. 

For example, function I implemented for this has the following inputs. You can implement it differently. Also, this is a situation where using Python class to carry out $N$-body calculations is very convenient. If you want to go this route, I'll be happy to provide guidance, if needed.  

    def nbody_integrate(nsteps, nsample, nprint, dt, x, v, mp, step, Etot, acc_func):
        """
        integrate equations of motions starting from the input vectors x, v, mp
        for nsteps with constant time step dt
        
        Parameters: 
        ------------
        
        nsteps - integer
                the number of steps to take during integration
        nsample - integer
                record physical variables of particles (x, v, and Epot) only each nsample-th step
        nprint  - integer
                print physical quantities every nprint-th step
        dt - float
            step size
        x, v, mp - vectors of floats
             coordinates, velocities and masses of particles
        step - python function
            name of a user-supplied function to compute step using a given stepping scheme
            it should take x, v, mp, dt, dth=dt/2 as inputs
            
        Etot - python function 
            name of a user-supplied function to compute total energy for input x, v
             it should take into account the difference in time slices of x and v if such
             difference exists
        acc_func  - python function
            name of a user-supplied function to compute mutual accelerations of particles
            and/or accelerations from an external potential
        
        Returns:
        -----------------
        tt - numpy float vector
            recorded orbit times
        xt, vt, Ett - numpy float vectors
            coordinates, velocities, and total energies of particles. 
         
        these arrays are initialized as follows: 
            tt  = np.empty(nsteps/nsample+2)
            xt  = np.empty(shape=(nsteps/nsample+2,) + np.shape(x))
            Ett = np.empty(nsteps/nsample+2)
            vt  = np.empty(shape=(nsteps/nsample+2,) + np.shape(x))

    """

Set up the calculation in such a way that units can be changed so you can use this function to integrate different physical systems easily by redefining units at the beginning of the calculation. This means that your evolution function should be written in abstract code units, which can be defined at the initialization. You can use the system of units with $G=1$ described in 01_orbit_integration notebook. 

**1b. (5 points)** Required tests and follow up questions: 
After you complete the function, test it by integrating a two body system with the Sun and the Earth with their true masses and with Earth on a circular orbit while the Sun is at rest initially. Run simulations using integration with the 1st order Euler and 2nd order leapfrog. An example of how this problem can be initialized  is provided in 01_orbit_integration, but you would need to initialize values in code units. Follow evolution for several tens of orbits (up to ~100) and make sure that at least for the leapfrog scheme the orbit stays circular during integration.  

Specifically, plot radius of the Earth orbit and its total energy as a function of time for the 1st order Euler and 2nd order leapfrog schemes.* How do results for these schemes compare? Discuss any obvious differences and their possible origin. (Note that at least one of these should show oscillating radius and energy but no net long term evolution of these quantities). Do results change/improve if you decrease step size significantly? 


***Note:*** The leapfrog scheme can be the DKD scheme described in the notebook, but you can also implement it in a simpler way by initializing coordinates at time $t_0-\Delta/2$ (using 1st order Euler scheme), while initializing velocities at $t_0$ and then stepping as follows:

\begin{eqnarray}
&D:& \ \ \vec{x}_{n+1/2} = \vec{x}_{n-1/2}+v_{n}\,\Delta t,\\
&K:& \ \ \vec{v}_{n+1}=\vec{v}_n + \vec{g}_{n+1/2}\, \Delta t, \\
\end{eqnarray}

Acceleration $\vec{g}_{n+1/2}$ can be computed after the $D$ part. You can find more information about this inS 1.4.5 in the notes).


#### Additional info and hints: 
The evolution code should consist of a loop stepping in time.
For each particle acceleration is computed by direct summation of forces from all other particles and use it to advance positions of particles. Thus, to compute acceleration of all particles one has to have two loops over particles (which is what makes this problem scale as $N^2$). 


**Note:** Use of Python classes is useful for this problem. If you feel comfortable with using classes I encourage you to try their use for this problem.  If you want to go this route, I'll be happy to provide guidance, if needed.  

**Note:** The second loop during step can be done via NumPy operation without an explicit loop. With some effort both loops in computation of accelerations for all $N$ particles can be replaced with NumPy operations. I encourage you to try to do try getting rid of the second or both loops using NumPy operations. If you will be able to avoid using inner loop using NumPy operation you will receive ***3 extra-credit points*** and if you will be able to avoid both you will get ***6 extra-credit points.*** Attempt this only after you get code working with a function computing accelerations using loops.  

### <font color='darkblue'> In the following choose either 2 or 3. Or do both for extra credit. 

Note that if for some reason you will not be able to make your code in the exercise 1 above to work properly. I can provide you with a function to carry out this exercise. You will receive a partial credit for exercise 1, depending on how much progress you've made towards complete implementation. 
    
### <font color='darkblue'>2. (10 points)  Integrating orbits of planets in the exoplanet systems GJ 876.

#### Background info:
GJ876 is a red dwarf star for which a series of precise radial motion measurements exists. Modelling of these motions indicates that it has at least two planets (quite likely three) orbiting around it. The two planets have periods of $\approx 60$ and $\approx 30$ days and are locked in a 2:1 mean motion resonance. 
Cartesian coordinates, velocities, and masses of the stars for one of the best fitting models are presented in Table 3 of <a href="http://adsabs.harvard.edu/abs/2005ApJ...622.1182L">Laughlin et al. 2005</a> and are available in this <a href="https://github.com/a-kravtsov/a330f17/blob/master/data/gj876.dat">file.</a> The code snippet below reads it.
Let's see if the 2nd order symplectic integrators we've discussed are good enough to reproduce the main results of this paper. 

#### Required tests and follow up questions: 
Initialize the three body system (star and two planets) with these coordinates and velocities as initial conditions, choosing suitable units: e.g., 1 AU as the unit of distance and $M_{\odot}$ as mass unit, day as a unit of time. Integrate them forward for 3320 days with the $N$-body code you implemented in 2.

Output positions and velocities of the planets and the star every half day from the start. Plot positions of the planets at the half day intervals from the start and compare to the figure 1 in Laughlin et al. 2005. Plot the radial velocity of the star (velocity in the 1D velocity in the plane of the planet orbits) as a function of time. Compare to Fig 2 in Laughlin et al. Can you reproduce the figures? Comment on what these figures show. 

In [3]:
# reading initial planet positions from Table 3 of Laughlin et al. 2005
import numpy as np 

data_file = 'https://astro.uchicago.edu/~andrey/classes/a211/data/gj876.dat'
name = np.loadtxt(data_file, usecols=[0], unpack=True, dtype=str)
mp, xp, yp, zp, vxp, vyp, vzp = np.loadtxt(data_file, usecols=(1,2,3,4,5,6,7), unpack=True)
print(name, mp, xp, yp, zp, vxp, vyp, vzp)

['Star' 'Planetc' 'Planetb'] [6.36515181e+32 1.13341374e+30 3.59700414e+30] [ 0.00000000e+00 -1.37393700e+12  2.89833447e+12] [ 0.0000000e+00  6.6185776e+11 -1.3485766e+12] [0. 0. 0.] [   -3974.15664 -2532174.78     1501141.65   ] [   -9012.47643 -5262209.95     3252940.14   ] [0. 0. 0.]


### <font color='darkblue'>3. (10 points) Long-term integration of planet orbits in the solar system. 
Integrate orbits of planets in the solar system over significant period of time (at least tens of thousands of orbits). Use the coordinates and velocities of the solar system planets that can be seen as shown below.  These are the actual accurate positions and velocities of the planets in the recent past. The code snippet below shows how to read them. Evolve the Sun and the planets using the $N$-body evolution code you've implemented for as long as you can. 

#### Required tests and follow up questions:
a. Monitor total energy (sum of the kinetic and potential energies for all particles) of the entire system (including the Sun) and distances of the planets to the Sun. Plot these as a function of time for suitable time intervals (say outputing every 10 years). Try integrating for 1000 or 10000 yrs first and examine the behavior of total energy and distances to the Sun as a function of time. If the behavior is stable, try integrating for much longer periods of time. Comment on the stability of the solar system based on results of your experiment. Was it possible for you to explore stability over a billion year time scale? 

b. Plot $x$ and $y$ positions of the Sun as a function of time (i.e., its motion in the solar plane) for a period of time that would allow to see how the Sun moves (i.e., not over-crowing the plot). You will need to zoom-in to a region very close to Sun's position. What do you see? Comment on what's going on. Take a look at the orbit of Mercury - the planet with the largest ellipticity for a few orbital periods. Does the phase of the ellipse stay the same? Comment on the result. How would you explain it?


In [2]:
import numpy as np

# read positions of the Sun and 9 planets at a moment in not so distant past...
# units: [positions]=[AU] [velocities]=[AU/day]
# data for the initial positions and velocities of planets
data_file = 'https://astro.uchicago.edu/~andrey/classes/a211/data/planets.dat'
# read planet names into array name
name = np.loadtxt(data_file, usecols=[0], unpack=True, dtype=str)
# read planet initial positions and velocities
mp, xp, yp, zp, vxp, vyp, vzp = np.loadtxt(data_file, usecols=(1,2,3,4,5,6,7), unpack=True)
print(name, mp, xp, yp, zp, vxp, vyp, vzp)

['MERCURY' 'VENUS' 'EARTH' 'MARS' 'JUPITER' 'SATURN' 'URANUS' 'NEPTUNE'
 'PLUTO' 'SUN'] [1.66013680e-07 2.44783834e-06 3.04043264e-06 3.22715145e-07
 9.54791938e-04 2.85885981e-04 4.36624404e-05 5.15138902e-05
 7.39644970e-09 1.00000000e+00] [-3.86050847e-01 -2.14010080e-01  5.35527126e-01  1.09184912e+00
 -3.49052855e+00 -5.58775550e+00  1.93504968e+01  2.74781931e+01
  7.26951759e+00  2.66834539e-03] [-1.55568598e-01 -6.93820243e-01  8.31190625e-01 -8.55123986e-01
  3.99373810e+00 -8.21991042e+00  5.09863735e+00 -1.19655388e+01
 -3.19225425e+01 -1.06975299e-03] [ 2.29070985e-02  2.87678885e-03 -1.62113878e-04 -4.47612425e-02
  6.14420124e-02  3.65307464e-01 -2.31755357e-01 -3.86855636e-01
  1.31313636e+00 -1.33058601e-04] [ 4.57643579e-03  1.91708282e-02 -1.47681412e-02  9.17276723e-03
 -5.77186746e-03  4.30968034e-03 -1.03085015e-03  1.23196567e-03
  3.12406160e-03  4.25486360e-06] [-2.49214023e-02 -6.11721925e-03  9.22268544e-03  1.22155851e-02
 -4.61038628e-03 -3.15154738e-03  3.6